## Text Anlaysis Flow
##### 1. Data Extraction using Beautiful scoop and creating seperate text file for each url (containing only article heading and content)
#####    each text file be named as the name of its url_id and stored in a directory "aritcles"
##### 2. From given master dictionary containing positive and negative words for text analysis and stopwords . Creating sets for each of them    #####  #####    seperately 
##### 3. Text analysis is divided into two modules:
#####    a. Analysis pre-stopwords removal 
#####    b. Analysis post-stopwords          Reason for dividing the text analysis into two divison was:
#####                                        1. Some metrics needed to be calculated pre stop words removal
#####                                        2. For ease of functions
##### 4. First part calculate :::    1. POSITIVE_SCORE  2.NEGATIVE_SCORE  3.Syllable_Count   4.num_words  5.Polartiy_Score  6.Subjectivity_Score  
##### 5. Second part calculate :::   1.avg_sentence_length  2.percentage_complex_words 3.fog  4.Average_Number_of_Words_Per_Sentence  
#####                                5.complex_word_count 6. Average_word_length  7.num_personal_pronouns  
##### 6. Merging the two data frames and updating the Output data file.                                

In [24]:
import pandas as pd
import requests
import nltk
import os
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict
import string

In [2]:
nltk.download('punkt')
nltk.download('punkt')
nltk.download('cmudict')

[nltk_data] Downloading package punkt to C:\Users\SK
[nltk_data]     SHARMA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\SK
[nltk_data]     SHARMA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to C:\Users\SK
[nltk_data]     SHARMA\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [32]:
excel_file = 'input.xlsx'
df = pd.read_excel(excel_file)
df.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


### Data Extraction from the URLS using Beautifulsoup

In [9]:
if 'URL_ID' not in df.columns or 'URL' not in df.columns:
    raise ValueError("Excel file must contain 'url_id' and 'url' columns")

In [10]:
os.makedirs('articles', exist_ok=True)

In [11]:
def fetch_article(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')

        # Assuming the title is in a <title> or <h1> tag
        title = soup.find('title') or soup.find('h1')
        if title:
            title = title.get_text(strip=True)
        else:
            title = "No Title Found"

        content_div = soup.find('div', class_='article-content') or soup.find('article')
        if content_div:
            paragraphs = content_div.find_all('p')
            content = '\n'.join([para.get_text(strip=True) for para in paragraphs])
        else:
            content = "No Content Found"

        return title, content
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None, None


for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    title, content = fetch_article(url)
    if title and content:
      
        file_name = f'articles/{url_id}.txt'
        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(f"Title: {title}\n\n{content}")

    print(f"Processed {url_id}: {url}")

print("Finished fetching articles.")

Processed blackassign0001: https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/
Processed blackassign0002: https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/
Processed blackassign0003: https://insights.blackcoffer.com/internet-demands-evolution-communication-impact-and-2035s-alternative-pathways/
Processed blackassign0004: https://insights.blackcoffer.com/rise-of-cybercrime-and-its-effect-in-upcoming-future/
Processed blackassign0005: https://insights.blackcoffer.com/ott-platform-and-its-impact-on-the-entertainment-industry-in-future/
Processed blackassign0006: https://insights.blackcoffer.com/the-rise-of-the-ott-platform-and-its-impact-on-the-entertainment-industry-by-2040/
Processed blackassign0007: https://insights.blackcoffer.com/rise-of-cyber-crime-and-its-effects/
Processed blackassign0008: https://insights.blac

### Creating given Stopwords set 

In [12]:
stop_words = set()
for files in os.listdir('F:\JUPYTER NOTEBOOK\StopWords'):
  with open(os.path.join('F:\JUPYTER NOTEBOOK\StopWords',files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

### Creating given Positive and Negative Word sets for the text analysis

In [13]:
positive_words = set()
negative_words=set()
with open('psw.txt', 'r', encoding='utf-8') as f:
    for line in f:
        positive_words.add(line.strip())
with open('neg.txt','r',encoding='ISO-8859-1') as f:
    for line in f :
        negative_words.add(line.strip())

### Text analysis

In [25]:
def count_positive_words(text, positive_words):
    tokens = word_tokenize(text)
    countp = sum(1 for token in tokens if token.lower() in positive_words)
    return countp

def count_negative_words(text, negative_words):
    tokens = word_tokenize(text)
    count = sum(-1 for token in tokens if token.lower() in negative_words) * -1
    return count

def count_syllables(text):
    """Count syllables in a word using the CMU Pronouncing Dictionary."""
    words = word_tokenize(text)
    total_syllables = 0
    for word in words:
        if word.lower() in prondict:
            
            syllables = [len(list(y for y in x if y[-1].isdigit())) for x in prondict[word.lower()] if not (word.endswith('es') or word.endswith('ed'))]
            total_syllables += max(syllables) if syllables else 0
    return total_syllables

stop_words = set(stopwords.words('english'))
def clean_and_count_words(text):
   
    words = word_tokenize(text)
    
    
    cleaned_words = [word for word in words if word.lower() not in stop_words and word not in string.punctuation]
    

    num_words = len(cleaned_words)
    
  
    cleaned_text = ' '.join(cleaned_words)
    
    return cleaned_text, num_words


def main():
    articles_dir = 'articles'
    results=[]
    error_files = []
  
    for text_file in os.listdir(articles_dir):
        positive_word_count = 0  # Reset positive word count
        negative_word_count = 0  # Reset negative word count
        
        file_path = os.path.join(articles_dir, text_file)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        
        url_id = text_file
        positive_word_count = count_positive_words(text, positive_words)
        negative_word_count = count_negative_words(text, negative_words)
        syllable_count = count_syllables(text)  
        cleaned_text, num_words = clean_and_count_words(text)
        
        results.append({
            'URL_ID': url_id,
            'POSITIVE_SCORE': positive_word_count,
            'NEGATIVE_SCORE': negative_word_count,
            'Syllable_Count': syllable_count,
            'num_words':num_words
        })
 
    results_df = pd.DataFrame(results)
    
    results_df['Polartiy_Score']= (results_df['POSITIVE_SCORE'] - results_df['NEGATIVE_SCORE'])/ ((results_df['POSITIVE_SCORE']+results_df['NEGATIVE_SCORE']) + 0.000001)
    results_df['Subjectivity_Score']=(results_df['POSITIVE_SCORE'] + results_df['NEGATIVE_SCORE'])/(num_words +0.000001)
    print(results_df)
    return results_df

if __name__ == '__main__':
    results_df=main()
        

                 URL_ID  POSITIVE_SCORE  NEGATIVE_SCORE  Syllable_Count  \
0   blackassign0001.txt               6               1             299   
1   blackassign0002.txt              56              29            1501   
2   blackassign0003.txt              38              24            1414   
3   blackassign0004.txt              38              75            1282   
4   blackassign0005.txt              21               8             707   
..                  ...             ...             ...             ...   
93  blackassign0096.txt              29              57            1090   
94  blackassign0097.txt              25              35             740   
95  blackassign0098.txt               1               0             188   
96  blackassign0099.txt              16               3             416   
97  blackassign0100.txt              31              55             801   

    num_words  Polartiy_Score  Subjectivity_Score  
0         175        0.714286            0.0144

#### Removing Stopwords and updating each file

In [14]:
def remove_stopwords(text, stopwords):
    return ' '.join(word for word in text.split() if word.lower() not in stopwords)
def main():
    articles_dir = 'articles'
    for text_file in os.listdir(articles_dir):
        file_path = os.path.join(articles_dir, text_file)
        if os.path.isfile(file_path) and text_file.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
            
            cleaned_text = remove_stopwords(text, stop_words)
            
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(cleaned_text)
            
            print(f"Processed {text_file}")

if __name__ == '__main__':
    main()

Processed blackassign0001.txt
Processed blackassign0002.txt
Processed blackassign0003.txt
Processed blackassign0004.txt
Processed blackassign0005.txt
Processed blackassign0006.txt
Processed blackassign0007.txt
Processed blackassign0008.txt
Processed blackassign0009.txt
Processed blackassign0010.txt
Processed blackassign0011.txt
Processed blackassign0012.txt
Processed blackassign0013.txt
Processed blackassign0014.txt
Processed blackassign0015.txt
Processed blackassign0016.txt
Processed blackassign0017.txt
Processed blackassign0018.txt
Processed blackassign0019.txt
Processed blackassign0020.txt
Processed blackassign0021.txt
Processed blackassign0022.txt
Processed blackassign0023.txt
Processed blackassign0024.txt
Processed blackassign0025.txt
Processed blackassign0026.txt
Processed blackassign0027.txt
Processed blackassign0028.txt
Processed blackassign0029.txt
Processed blackassign0030.txt
Processed blackassign0031.txt
Processed blackassign0032.txt
Processed blackassign0033.txt
Processed 

In [22]:

prondict = cmudict.dict()

def count_syllables(word):
   
    if word.lower() in prondict:
      
        syllables = [len(list(y for y in x if y[-1].isdigit())) for x in prondict[word.lower()] if not (word.endswith('es') or word.endswith('ed'))]
        return max(syllables) if syllables else 0
    else:
      
        return 0

def text_analytics(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
   
    words = word_tokenize(text)
    

    personal_pronouns = ['I', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves']
    num_personal_pronouns = sum(1 for word in words if word.lower() in personal_pronouns)

   
    num_complex_words = sum(1 for word in words if count_syllables(word) > 1)
    
    percentage_complex_words = (num_complex_words / len(words)) * 100 if words else 0
    
    sentences = sent_tokenize(text)
    
    num_sentences = len(sentences)
    total_sentence_length = sum(len(word_tokenize(sentence)) for sentence in sentences)
    avg_sentence_length = total_sentence_length / num_sentences if num_sentences else 0
    fog = 0.4 * (avg_sentence_length + percentage_complex_words)
    Average_Number_of_Words_Per_Sentence = len(words) / len(sentences)
    complex_word_count = sum(1 for word in words if count_syllables(word) > 1)
    Average_word_length = sum(len(word) for word in words) / len(words)

    return {
        'avg_sentence_length': avg_sentence_length,
        'percentage_complex_words': percentage_complex_words,
        'fog': fog,
        'Average_Number_of_Words_Per_Sentence': Average_Number_of_Words_Per_Sentence,
        'complex_word_count': complex_word_count,
        'Average_word_length': Average_word_length,
        'num_personal_pronouns': num_personal_pronouns,
    }

def  process_files_in_directory (article_dir):
    file_statistics = []
    for filename in os.listdir(article_dir):
        file_path = os.path.join(article_dir, filename)
        if os.path.isfile(file_path):
            stats = text_analytics(file_path)
            stats['filename'] = filename
            file_statistics.append(stats)
            print(f"Processed {filename}: {stats}")
    return file_statistics


article_dir = 'articles'


file_statistics = process_files_in_directory(article_dir)
res_df = pd.DataFrame(file_statistics)
res_df['URL_ID'] = res_df['filename']
res_df.drop(columns=['filename'], inplace=True)
print(res_df)

Processed blackassign0001.txt: {'avg_sentence_length': 8.72, 'percentage_complex_words': 44.4954128440367, 'fog': 21.28616513761468, 'Average_Number_of_Words_Per_Sentence': 8.72, 'complex_word_count': 97, 'Average_word_length': 5.385321100917431, 'num_personal_pronouns': 0, 'filename': 'blackassign0001.txt'}
Processed blackassign0002.txt: {'avg_sentence_length': 13.2, 'percentage_complex_words': 47.878787878787875, 'fog': 24.431515151515153, 'Average_Number_of_Words_Per_Sentence': 13.2, 'complex_word_count': 474, 'Average_word_length': 6.097979797979798, 'num_personal_pronouns': 2, 'filename': 'blackassign0002.txt'}
Processed blackassign0003.txt: {'avg_sentence_length': 14.089285714285714, 'percentage_complex_words': 56.27376425855514, 'fog': 28.145219989136343, 'Average_Number_of_Words_Per_Sentence': 14.089285714285714, 'complex_word_count': 444, 'Average_word_length': 6.845373891001268, 'num_personal_pronouns': 1, 'filename': 'blackassign0003.txt'}
Processed blackassign0004.txt: {'av

In [41]:
merged_df = results_df.merge(res_df, on='URL_ID', how='inner')
merged_df['URL_ID'] = results_df['URL_ID'].str.split('.').str[0]
new_df = merged_df.merge(df, on='URL_ID', how='inner')

In [40]:
desired_order = ['URL_ID', 'URL', 'POSITIVE_SCORE', 'NEGATIVE_SCORE', 'Syllable_Count', 'num_words', 'Polartiy_Score',
                 'Subjectivity_Score', 'avg_sentence_length', 'percentage_complex_words', 'fog', 
                 'Average_Number_of_Words_Per_Sentence', 'complex_word_count', 'Average_word_length', 
                 'num_personal_pronouns']

df = new_df[desired_order]
df.head()

,URL_ID,URL,POSITIVE_SCORE,NEGATIVE_SCORE,Syllable_Count,num_words,Polartiy_Score,Subjectivity_Score,avg_sentence_length,percentage_complex_words,fog,Average_Number_of_Words_Per_Sentence,complex_word_count,Average_word_length,num_personal_pronouns
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,6,1,299,175,0.714286,0.014493,8.720000,44.495413,21.286165,8.720000,97,5.385321,0
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,56,29,1501,788,0.317647,0.175983,13.200000,47.878788,24.431515,13.200000,474,6.097980,2
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,38,24,1414,637,0.225806,0.128364,14.089286,56.273764,28.145220,14.089286,444,6.845374,1
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,38,75,1282,617,-0.327434,0.233954,15.480000,52.971576,27.380630,15.480000,410,6.607235,0
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,21,8,707,376,0.448276,0.060041,11.717949,50.328228,24.818471,11.717949,230,6.356674,0


In [42]:
output_file = 'output.xlsx'
column_mapping = {
     'URL':'URL',
    'POSITIVE_SCORE': 'POSITIVE SCORE',
    'NEGATIVE_SCORE': 'NEGATIVE SCORE',
    'Polartiy_Score': 'POLARITY SCORE',
    'Subjectivity_Score': 'SUBJECTIVITY SCORE',
    'avg_sentence_length': 'AVG SENTENCE LENGTH',
    'percentage_complex_words': 'PERCENTAGE OF COMPLEX WORDS',
    'fog': 'FOG INDEX',
    'Average_Number_of_Words_Per_Sentence': 'AVG NUMBER OF WORDS PER SENTENCE',
    'complex_word_countT': 'COMPLEX WORD COUNT',
    'num_words': 'WORD COUNT',
    'Syllable_Count': 'SYLLABLE PER WORD',
    'num_personal_pronouns': 'PERSONAL PRONOUNS',
    'Average_word_length': 'AVG WORD LENGTH'
}


df.rename(columns=column_mapping, inplace=True)

with pd.ExcelWriter(output_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    
    df.to_excel(writer, sheet_name='Sheet1', index=False)
